# Thème 4 – Le vélo au service des espaces du quotidien
**Analyse carto :** Croisement base d’équipement INSEE et point rouge baromètre 
- Identifier le % de point rouge à – de 50 m d’un établissement d’enseignement, de santé et de commerces  

> L’accessibilité aux communes pourvoyeuses de services et d’équipements est encore trop complexe en France 


## Python stuff

In [34]:
import pandas as pd
import geopandas as gpd
from s3_utils import S3Manager
import matplotlib.pyplot as plt
from scipy.stats import fisher_exact

_ = plt.style.use("ggplot")

In [35]:
# Chargement des éléments de connexion sur S3
s3_manager = S3Manager()
bucket = "fub-s3"

## 1 - Récupération et préparation des données

In [36]:
# Import des données sources
gdf = s3_manager.load_geojson_from_s3(bucket, "data/DFG/2025/data_geo/données-carto-2025-06-03.geojson")
print(gdf.crs)
gdf.head(2)

✅ Fichier chargé et converti en GeoDataFrame avec succès
EPSG:4326


,uid,cat,uid_reponse,description,geometry
0,dd441ba7-d3a7-490e-8cfe-1d34e4a09311,58,ffde2d2e-fa32-4bbb-a7f2-f94300dfb19d,Route trop étroite et dangereuse,POINT (4.47109 47.1244)
1,63ef8e40-8be8-4362-b9ea-5b38a0347ef3,59,ffde2d2e-fa32-4bbb-a7f2-f94300dfb19d,Maison de santé,POINT (4.48483 47.13117)


In [37]:
url_contour_france = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
contour_france = gpd.read_file(url_contour_france)

In [38]:
gdf = gdf.to_crs(contour_france.crs)
gdf_filtre = gpd.sjoin(gdf, contour_france, predicate="within", how="inner")
gdf_filtre = gdf_filtre.drop(columns=["index_right"])

In [39]:
# Import des données d'enrichissements depuis S3
s3_manager.download_from_s3(
    bucket, 
    "data/DFG/2025/data_geo/externe/4C-buildingref-france-bpe-all-geolocated.fgb", 
    "temp/buildingref-france-bpe-all-geolocated.fgb"
)
equipements = gpd.read_file("temp/buildingref-france-bpe-all-geolocated.fgb")
print(equipements.crs)
equipements.head(2)

✅ Fichier téléchargé depuis S3 : temp/buildingref-france-bpe-all-geolocated.fgb
EPSG:4326


,reg_code,dep_code,com_arm_code,iris_code,year,equipment_code,geocode_quality,equipment_name,category,reg_name,dep_name,epci_name,com_arm_name,com_arm_area_code,epci_code,geometry
0,93,13,13026,130260104,NaT,D603,Acceptable,ADULTES HANDICAPÉS : ACCUEIL/HÉBERGEMENT,Santé,Provence-Alpes-Côte d'Azur,Bouches-du-Rhône,Métropole d'Aix-Marseille-Provence,Châteauneuf-les-Martigues,FXX,200054807,POINT (5.14153 43.39474)
1,93,83,83049,830490102,NaT,D603,Bonne,ADULTES HANDICAPÉS : ACCUEIL/HÉBERGEMENT,Santé,Provence-Alpes-Côte d'Azur,Var,CC Méditerranée Porte des Maures,Cuers,FXX,200027100,POINT (6.08587 43.25823)


In [40]:
url_contour_france = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
contour_france = gpd.read_file(url_contour_france)

In [41]:
equipements = equipements.to_crs(contour_france.crs)
equipements = gpd.sjoin(equipements, contour_france, predicate="within", how="inner")
equipements = equipements.drop(columns=["index_right", "code", "nom"])

In [50]:
equipements = equipements[[
    'equipment_name', 'category', 'com_arm_name', 'geometry'
]].copy()

In [84]:
equipements.groupby('com_arm_name').size().reset_index(name="Nombre d'équipement").sort_values(by="Nombre d'équipement", ascending=False).head(10)

,com_arm_name,Nombre d'équipement
21405,Toulouse,6074
14425,Nice,5301
2523,Bordeaux,4082
13665,Montpellier,3853
20856,Strasbourg,3495
14229,Nantes,3467
15172,Paris 15e Arrondissement,3161
15173,Paris 16e Arrondissement,2966
11556,Lille,2957
15174,Paris 17e Arrondissement,2793


## 2 - Identifier le % de point rouge à – de 50 m d’un établissement d’enseignement, de santé et de commerces

### 2.1 - Statistique

In [51]:
# Reprojection spatiale
gdf_2154 = gdf_filtre.to_crs(2154)
equipements_2154 = equipements.to_crs(2154)

In [52]:
prox_equip = gpd.sjoin_nearest(
    gdf_2154,
    equipements_2154,
    how="left",
    max_distance=50,
    lsuffix="velo",
    rsuffix="equipement",
    distance_col="dist_m"
)
prox_equip_e = prox_equip.copy()

In [78]:
print(f"Nombre de point rouge : {gdf_2154[gdf_2154['cat'] == 58].shape[0]}")
prox_58 = prox_equip_e.loc[
        (prox_equip_e["cat"] == 58) &
        prox_equip_e["index_equipement"].notnull()
].shape[0]
print(f"Nombre de point rouge à moins de 50m d'un équipement : {prox_58}")
pct_points_couverts = 100 * prox_58 / gdf_2154[gdf_2154['cat'] == 58].shape[0]
print(f"Pourcentage de point rouge à moins de 50m d'un équipement : {pct_points_couverts:.2f}%")

Nombre de point rouge : 578073
Nombre de point rouge à moins de 50m d'un équipement : 248273
Pourcentage de point rouge à moins de 50m d'un équipement : 42.95%


### 2.2 - Test d'hypothèse

In [69]:
n58_total     = gdf_2154[gdf_2154["cat"] == 58].shape[0]
n58_covered   = prox_equip_e.loc[
                  (prox_equip_e["cat"] == 58) &
                  prox_equip_e["index_equipement"].notnull()
               ].shape[0]
n58_notcov    = n58_total - n58_covered

In [70]:
nrest_total   = gdf_2154[gdf_2154["cat"] != 58].shape[0]
nrest_covered = prox_equip_e.loc[
                  (prox_equip_e["cat"] != 58) &
                  prox_equip_e["index_equipement"].notnull()
               ].shape[0]
nrest_notcov  = nrest_total - nrest_covered

In [71]:
table = [[n58_covered,   n58_notcov],
         [nrest_covered, nrest_notcov]]

In [72]:
odds, p = fisher_exact(table, alternative="two-sided")
print(table)
print(f"Odds-ratio : {odds:.3f}")
print(f"p-value    : {p:.4g}")

[[248273, 329800], [287259, 241775]]
Odds-ratio : 0.634
p-value    : 0


## 3 - Conclusion

L’analyse spatiale révèle que seuls 42,95 % des points rouges (lieux problématiques identifiés par le baromètre vélo) se situent à moins de 50 m d’un établissement d’enseignement, de santé ou de commerce, contre une proportion significativement plus élevée pour les autres points (test exact de Fisher, OR = 0,63, p < 0,001).
Ces résultats montrent que les zones jugées les plus problématiques pour les cyclistes se trouvent, en moyenne, plus éloignées des services et équipements du quotidien.

-- END --